In [1]:
%config IPCompleter.greedy=True

# FIFA 21 Player Data Process
## Dataset
* [FIFA 21 players dataset](https://www.kaggle.com/datasets/yagunnersya/fifa-21-messy-raw-dataset-for-cleaning-exploring)

## Guiding Questions:
* Do the height and weight columns have the appropriate data types?
* Can you separate the joined column into year, month and day columns?
* Can you clean and transform the value, wage and release clause columns into columns of intergers?
* How can you remove the newline characters from the Hits column?
* Should you separate the Team & Contract column into separate team and contract columns?

In [122]:
import pandas as pd
import re
import matplotlib.pyplot as plt

In [11]:
d = pd.read_csv(
    "dataset/fifa21_raw_data.csv",
    dtype={"Hits": "string"} # DtypeWarning: Columns (76) have mixed types.
)
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   photoUrl          18979 non-null  object
 1   LongName          18979 non-null  object
 2   playerUrl         18979 non-null  object
 3   Nationality       18979 non-null  object
 4   Positions         18979 non-null  object
 5   Name              18979 non-null  object
 6   Age               18979 non-null  int64 
 7   ↓OVA              18979 non-null  int64 
 8   POT               18979 non-null  int64 
 9   Team & Contract   18979 non-null  object
 10  ID                18979 non-null  int64 
 11  Height            18979 non-null  object
 12  Weight            18979 non-null  object
 13  foot              18979 non-null  object
 14  BOV               18979 non-null  int64 
 15  BP                18979 non-null  object
 16  Growth            18979 non-null  int64 
 17  Joined      

In [8]:
d.head()

,photoUrl,LongName,playerUrl,Nationality,Positions,Name,Age,↓OVA,POT,Team & Contract,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,https://cdn.sofifa.com/players/158/023/21_60.png,Lionel Messi,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,RW ST CF,L. Messi,33,93,93,\n\n\n\nFC Barcelona\n2004 ~ 2021\n\n,...,Medium,Low,5 ★,85,92,91,95,38,65,372
1,https://cdn.sofifa.com/players/020/801/21_60.png,C. Ronaldo dos Santos Aveiro,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,ST LW,Cristiano Ronaldo,35,92,92,\n\n\n\nJuventus\n2018 ~ 2022\n\n,...,High,Low,5 ★,89,93,81,89,35,77,344
2,https://cdn.sofifa.com/players/200/389/21_60.png,Jan Oblak,http://sofifa.com/player/200389/jan-oblak/210005/,Slovenia,GK,J. Oblak,27,91,93,\n\n\n\nAtlético Madrid\n2014 ~ 2023\n\n,...,Medium,Medium,3 ★,87,92,78,90,52,90,86
3,https://cdn.sofifa.com/players/192/985/21_60.png,Kevin De Bruyne,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,CAM CM,K. De Bruyne,29,91,91,\n\n\n\nManchester City\n2015 ~ 2023\n\n,...,High,High,4 ★,76,86,93,88,64,78,163
4,https://cdn.sofifa.com/players/190/871/21_60.png,Neymar da Silva Santos Jr.,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,LW CAM,Neymar Jr,28,91,91,\n\n\n\nParis Saint-Germain\n2017 ~ 2022\n\n,...,High,Medium,5 ★,91,85,86,94,36,59,273


In [14]:
d.columns

Index(['photoUrl', 'LongName', 'playerUrl', 'Nationality', 'Positions', 'Name',
       'Age', '↓OVA', 'POT', 'Team & Contract', 'ID', 'Height', 'Weight',
       'foot', 'BOV', 'BP', 'Growth', 'Joined', 'Loan Date End', 'Value',
       'Wage', 'Release Clause', 'Attacking', 'Crossing', 'Finishing',
       'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill', 'Dribbling',
       'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control', 'Movement',
       'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
       'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots',
       'Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Defending', 'Marking', 'Standing Tackle',
       'Sliding Tackle', 'Goalkeeping', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Total Stats',
       'Base Stats', 'W/F', 'SM', 'A/W', 'D/W', 'IR', 'PAC', 'SHO', 'PAS',
       'DRI', 'DEF', 'PHY', 'Hits

## Do the height and weight columns have the appropriate data types?

In [18]:
d[["Weight", "Height"]].head()

,Weight,Height
0,159lbs,"5'7"""
1,183lbs,"6'2"""
2,192lbs,"6'2"""
3,154lbs,"5'11"""
4,150lbs,"5'9"""


In [19]:
d[["Weight", "Height"]].dtypes

Weight    object
Height    object
dtype: object

In [47]:
d["weight_kg"] = (
    d["Weight"].str
    .extract(r'(\d+)\s*lbs')
    .astype(float)
    .mul(0.453592)
    .round()
)

In [35]:
d["height_cm"] = (
    d["Height"].str
    .extract(r'(\d+)\'\s*(\d+)"')
    .astype(float)
    .mul([12*2.54, 2.54])
    .sum(axis=1)
)

In [48]:
d[["weight_kg", "height_cm"]].head()

,weight_kg,height_cm
0,72.0,170.18
1,83.0,187.96
2,87.0,187.96
3,70.0,180.34
4,68.0,175.26


##　Can you separate the Joined column into year, month and day columns?

In [51]:
d[["Joined"]].head()

,Joined
0,"Jul 1, 2004"
1,"Jul 10, 2018"
2,"Jul 16, 2014"
3,"Aug 30, 2015"
4,"Aug 3, 2017"


In [58]:
d["joined_date"] = pd.to_datetime(d["Joined"], format="%b %d, %Y")

In [62]:
d["joined_year"] = d["joined_date"].dt.year
d["joined_month"] = d["joined_date"].dt.month
d["joined_day"] = d["joined_date"].dt.day

In [65]:
d[["Joined", "joined_date", "joined_year","joined_month", "joined_day"]].head()

,Joined,joined_date,joined_year,joined_month,joined_day
0,"Jul 1, 2004",2004-07-01,2004,7,1
1,"Jul 10, 2018",2018-07-10,2018,7,10
2,"Jul 16, 2014",2014-07-16,2014,7,16
3,"Aug 30, 2015",2015-08-30,2015,8,30
4,"Aug 3, 2017",2017-08-03,2017,8,3


## Can you clean and transform the value, wage and release clause columns into columns of intergers?

In [75]:
d[["Wage", "Value", "Release Clause"]].head()

,Wage,Value,Release Clause
0,€560K,€67.5M,€138.4M
1,€220K,€46M,€75.9M
2,€125K,€75M,€159.4M
3,€370K,€87M,€161M
4,€270K,€90M,€166.5M


In [ ]:
# Knowing if there are different unit

In [97]:
print("Player wage with K unit: ", d[["Wage"]][d["Wage"].str.contains("K")].shape[0])
print("Player wage with M unit: ", d[["Wage"]][d["Wage"].str.contains("M")].shape[0])
print("Player wage without unit: ", d[["Wage"]][
      ~(d["Wage"].str.contains("M")
        | d["Wage"].str.contains("K"))
    ].shape[0])

Player wage with K unit:  14824
Player wage with M unit:  0
Player wage without unit:  4155


In [96]:
print("Player value with K unit: ", d[["Value"]][d["Value"].str.contains("K")].shape[0])
print("Player value with M unit: ", d[["Value"]][d["Value"].str.contains("M")].shape[0])
print("Player value without unit: ", d[["Value"]][
      ~(d["Value"].str.contains("M")
        | d["Value"].str.contains("K"))
    ].shape[0])

Player value with K unit:  11944
Player value with M unit:  6786
Player value without unit:  249


In [110]:
def handle_amount(s: str)->int:
    pattern = r'€(\d+(\.\d+)?)[A-Z]*'
    num = float(re.search(pattern, s).group(1))
    if "M" in s:
        num *= 1000000
    elif "K" in s:
        num *= 1000
    return int(num)

In [111]:
d["value_eur"] = d["Value"].apply(handle_amount)
d["wage_eur"] = d["Wage"].apply(handle_amount)
d["release_zlause_eur"] = d["Release Clause"].apply(handle_amount)

In [113]:
d[[
    "Value", "value_eur", 
    "Wage", "wage_eur", 
    "Release Clause", "release_zlause_eur"
]].head()

,Value,value_eur,Wage,wage_eur,Release Clause,release_zlause_eur
0,€67.5M,67500000,€560K,560000,€138.4M,138400000
1,€46M,46000000,€220K,220000,€75.9M,75900000
2,€75M,75000000,€125K,125000,€159.4M,159400000
3,€87M,87000000,€370K,370000,€161M,161000000
4,€90M,90000000,€270K,270000,€166.5M,166500000


How can you remove the newline characters from the Hits column?

In [156]:
d["Hits"].iloc[0]

'\n372'

In [154]:
d["Hits"].iloc[99]

'\n1.3K'

In [166]:
def handle_hits(s: str)->int:
    tmp = None
    if s:
        tmp = s.replace("\n", "")
        if "K" in tmp:
            tmp = int(float(tmp.replace("K", "")) * 1000)
        else:
            tmp = int(tmp)
        return tmp
    else:
        return tmp
d["hits"] = d["Hits"].apply(handle_hits)

In [163]:
d[["Hits", "hits"]].head()

,Hits,hits
0,372,372
1,344,344
2,86,86
3,163,163
4,273,273


In [164]:
d["hits"].iloc[99]

1300

Should you separate the Team & Contract column into separate team and contract columns?

In [168]:
d[["Team & Contract"]].head()

,Team & Contract
0,\n\n\n\nFC Barcelona\n2004 ~ 2021\n\n
1,\n\n\n\nJuventus\n2018 ~ 2022\n\n
2,\n\n\n\nAtlético Madrid\n2014 ~ 2023\n\n
3,\n\n\n\nManchester City\n2015 ~ 2023\n\n
4,\n\n\n\nParis Saint-Germain\n2017 ~ 2022\n\n


In [208]:
d["Team & Contract"].iloc[205]

'\n\n\n\nTottenham Hotspur\nJun 30, 2021 On Loan\n\n'

In [209]:
d["team"] = d["Team & Contract"].str.split("\n").str[4]
d["contract"] = d["Team & Contract"].str.split("\n").str[5]

In [211]:
d[[ "team",  "contract"]].head()

,team,contract
0,FC Barcelona,2004 ~ 2021
1,Juventus,2018 ~ 2022
2,Atlético Madrid,2014 ~ 2023
3,Manchester City,2015 ~ 2023
4,Paris Saint-Germain,2017 ~ 2022


In [213]:
d["contract"].iloc[205]

'Jun 30, 2021 On Loan'

In [221]:
d.groupby("team").size()

team
                         238
1. FC Heidenheim 1846     30
1. FC Kaiserslautern      28
1. FC Köln                31
1. FC Magdeburg           28
                        ... 
Zamora FC                 28
Çaykur Rizespor           30
Örebro SK                 26
Östersunds FK             26
Śląsk Wrocław             27
Length: 682, dtype: int64